In [595]:
import pandas as pd
import numpy as np

In [596]:
temp = pd.read_csv('temperatureaulesETSAB2023_clean.csv')
co2 = pd.read_csv('co2aulesETSAB2023_clean.csv')
hum = pd.read_csv('humidityaulesETSAB2023_clean.csv')

In [597]:
temp.head()

Date  Aula  Temperatura    epoch  max_comfort  min_comfort
0  2023-01-01 00:00:00  A-11        17.80  heating           25           21
1  2023-01-01 01:00:00  A-11        17.70  heating           25           21
2  2023-01-01 02:00:00  A-11        17.60  heating           25           21
3  2023-01-01 03:00:00  A-11        17.50  heating           25           21
4  2023-01-01 04:00:00  A-11        17.45  heating           25           21

In [598]:
result1 = pd.merge(temp, co2, on=['Date','Aula'])
result1 = result1.drop(columns=['epoch'])
result = pd.merge(result1, hum, on=['Date','Aula'])
result = result.drop(columns=['planta', 'posicio'])
result.head()

Date  Aula  Temperatura  max_comfort  min_comfort    CO2  \
0  2023-01-01 00:00:00  A-11        17.80           25           21  492.5   
1  2023-01-01 01:00:00  A-11        17.70           25           21  511.0   
2  2023-01-01 02:00:00  A-11        17.60           25           21  522.0   
3  2023-01-01 03:00:00  A-11        17.50           25           21  564.0   
4  2023-01-01 04:00:00  A-11        17.45           25           21  521.0   

   outdoors_co2  max_co2  Humidity  min_humidity  max_humidity  
0           400     1000      55.0            30            60  
1           400     1000      55.0            30            60  
2           400     1000      55.0            30            60  
3           400     1000      54.5            30            60  
4           400     1000      54.0            30            60

In [599]:
def detect_excessive_temp(temp):
    return temp>25

def detect_toolow_temp(temp):
    return temp<20

def detect_excessive_co2(co2):
    return (co2>1000)

def detect_excessive_humidity(humidity):
    return (humidity>60 or humidity<30)

def detect_high_hum(hum):
    return hum>60

def detect_toolow_hum(hum):
    return hum<30

In [600]:
result['temp_excessiva_alta'] = result['Temperatura'].map(detect_excessive_temp)
result['temp_excessiva_baixa'] = result['Temperatura'].map(detect_toolow_temp)
result['temp_excessiva'] = result['temp_excessiva_alta'] + result['temp_excessiva_baixa']
result['co2_excessiu'] = result['CO2'].map(detect_excessive_co2)
result['humitat_excessiva'] = result['Humidity'].map(detect_excessive_humidity)
result['humitat_alta'] = result['Humidity'].map(detect_high_hum)
result['humitat_baixa'] = result['Humidity'].map(detect_toolow_hum)
df = result.copy()

In [601]:
result.head()

Date  Aula  Temperatura  max_comfort  min_comfort    CO2  \
0  2023-01-01 00:00:00  A-11        17.80           25           21  492.5   
1  2023-01-01 01:00:00  A-11        17.70           25           21  511.0   
2  2023-01-01 02:00:00  A-11        17.60           25           21  522.0   
3  2023-01-01 03:00:00  A-11        17.50           25           21  564.0   
4  2023-01-01 04:00:00  A-11        17.45           25           21  521.0   

   outdoors_co2  max_co2  Humidity  min_humidity  max_humidity  \
0           400     1000      55.0            30            60   
1           400     1000      55.0            30            60   
2           400     1000      55.0            30            60   
3           400     1000      54.5            30            60   
4           400     1000      54.0            30            60   

   temp_excessiva_alta  temp_excessiva_baixa  temp_excessiva  co2_excessiu  \
0                False                  True            True         False   
1                False                  True            True         False   
2                False                  True            True         False   
3                False                  True            True         False   
4                False                  True            True         False   

   humitat_excessiva  humitat_alta  humitat_baixa  
0              False         False          False  
1              False         False          False  
2              False         False          False  
3              False         False          False  
4              False         False          False

In [602]:
dies_festius = ['2023-09-25', '2023-10-12', '2023-11-01', '2023-12-06', '2023-12-07', '2023-12-08',
              '2023-12-23','2023-12-24','2023-12-25','2023-12-26','2023-12-27','2023-12-28','2023-12-29',
              '2023-12-30', '2023-12-31','2023-01-01','2023-01-02','2023-01-03','2023-01-04','2023-01-05',
              '2023-01-06','2023-01-07','2023-01-08','2023-01-09','2023-01-10','2023-04-01','2023-04-02',
              '2023-04-03','2023-04-04','2023-04-05','2023-04-06','2023-04-07','2023-04-08','2023-04-09',
              '2023-04-10','2023-05-01','2023-09-01','2023-09-02','2023-09-03','2023-09-04','2023-09-05',
               '2023-09-06','2023-09-07','2023-09-08','2023-09-09','2023-09-10','2023-09-11','2023-09-12']

def marcar_festius_seasonalplot(date):
    return (str(date).split(' ')[0] in dies_festius) or (str(date).split('-')[1] == '08') or (str(date).split('-')[1] == '07')
result['Date'] = pd.to_datetime(result['Date'], format='%Y-%m-%d %H:%M:%S')
result['Month'] = result['Date'].dt.month
result['Hour'] = result['Date'].dt.hour
result = result[(result['Hour']<=21) & (result['Hour']>=8)]
result['vacation'] = result['Date'].map(marcar_festius_seasonalplot)
result = result[result['vacation']==False]
result['day_of_week'] = result['Date'].dt.weekday
# Create a new column to indicate if the date falls on a weekend (Saturday or Sunday)
result['is_weekend'] = result['day_of_week'].isin([5, 6])
result = result[result['is_weekend']==False]
sourceQ1 = result[result['Month'].isin([9,10,11,12,1])]
sourceQ2 = result[result['Month'].isin([2,3,4,5,6])]
result2 = result.drop(columns=['Temperatura','Hour','max_comfort','min_comfort','CO2','outdoors_co2','max_co2','Humidity','min_humidity','max_humidity','is_weekend','day_of_week','vacation','Month'])
sourceQ1 = sourceQ1.drop(columns=['Temperatura','Hour','max_comfort','min_comfort','CO2','outdoors_co2','max_co2','Humidity','min_humidity','max_humidity','is_weekend','day_of_week','vacation','Month'])
sourceQ2 = sourceQ2.drop(columns=['Temperatura','Hour','max_comfort','min_comfort','CO2','outdoors_co2','max_co2','Humidity','min_humidity','max_humidity','is_weekend','day_of_week','vacation','Month'])
qualitat_aules = result2.groupby(['Aula']).sum().reset_index()
sourceQ1 = sourceQ1.groupby(['Aula']).sum().reset_index()
sourceQ2 = sourceQ2.groupby(['Aula']).sum().reset_index()

qualitat_aules.head()

Aula  temp_excessiva_alta  temp_excessiva_baixa  temp_excessiva  \
0  A-11                  690                   285             975   
1  A-12                  733                   391            1124   
2  A-13                  696                   392            1088   
3  A-14                  659                   569            1228   
4  A-21                  787                   249            1036   

   co2_excessiu  humitat_excessiva  humitat_alta  humitat_baixa  
0           103                406           136            270  
1           111                337           117            220  
2           161                301           112            189  
3           102                270            98            172  
4            58                386            72            314

In [603]:
def get_percentage(row, column):
    value = row[column]
    return round(value * 100 / 2600, 2)
columns_to_update = [
    'temp_excessiva',
    'temp_excessiva_alta',
    'temp_excessiva_baixa',
    'co2_excessiu',
    'humitat_excessiva',
    'humitat_alta',
    'humitat_baixa'
]

# Apply the function to each column
for column in columns_to_update:
    qualitat_aules[column] = qualitat_aules.apply(lambda row: get_percentage(row, column), axis=1)
    sourceQ1[column] = sourceQ1.apply(lambda row: get_percentage(row, column), axis=1)
    sourceQ2[column] = sourceQ2.apply(lambda row: get_percentage(row, column), axis=1)

aules = pd.DataFrame(data={'Aula': ['A-11','A-12','A-13','A-14','A-21','A-22','A-23','A-24','A-31','A-32',
'A-33','A-34','A-35','A-36','A-41','A-42','A-43','A-44','A-51','A-52','A-53','A-54','A-55','A-56','A-61','A-62'],
                               'planta': [1,1,1,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,5,5,5,5,5,5,6,6],
                               'posicio': [1,2,3,4,1,2,3,4,1,2,3,4,5,6,1,2,3,4,1,2,3,4,5,6,1,2]
})
qualitat_aules = pd.merge(qualitat_aules, aules, on=['Aula'])
sourceQ1 = pd.merge(sourceQ1, aules, on=['Aula'])
sourceQ2 = pd.merge(sourceQ2, aules, on=['Aula'])

qualitat_aules.to_csv('qualitat_aules_globalnovacation.csv')

In [604]:
sourceQ1.to_csv('qualitat_aules_Q1.csv')
sourceQ2.to_csv('qualitat_aules_Q2.csv')

In [605]:
result = df.copy()
result2 = result.drop(columns=['Temperatura','max_comfort','min_comfort','CO2','outdoors_co2','max_co2','Humidity','min_humidity','max_humidity'])
result2['Date'] = pd.to_datetime(result2['Date'], format='%Y-%m-%d %H:%M:%S')
result2['Month'] = result2['Date'].dt.month
qualitat_aules = result2.groupby(['Aula','Month']).sum().reset_index()

In [606]:
result2aula11 = result2[result2['Aula']=='A-11']
num_days_per_month = result2aula11.groupby('Month').count().reset_index()
num_days_per_month.head()
dict_dayspermonth = num_days_per_month.set_index('Month')['Date'].to_dict()
print(dict_dayspermonth)

{1: 744, 2: 672, 3: 743, 4: 720, 5: 744, 6: 720, 7: 744, 8: 744, 9: 720, 10: 751, 11: 720, 12: 744}


In [607]:
# TENIM 8760 MOSTRES PER CADA VARIABLE (24 H x 365 DIES) --> podem fer tant per cent en comptes de sum
# dividim entre 87.60 (aixi no cal multiplicar x 100 desp)
# HO CANVIO PER MES! --> canviem 87.6 per num samples per mes calculats a dalt

In [608]:
def get_percentage(row, column):
    month = row['Month']  # Assuming 'Date' column exists and is in datetime format
    value = row[column]
    return round(value * 100 / dict_dayspermonth[month], 2)
columns_to_update = [
    'temp_excessiva',
    'temp_excessiva_alta',
    'temp_excessiva_baixa',
    'co2_excessiu',
    'humitat_excessiva',
    'humitat_alta',
    'humitat_baixa'
]

# Apply the function to each column
for column in columns_to_update:
    qualitat_aules[column] = qualitat_aules.apply(lambda row: get_percentage(row, column), axis=1)

In [609]:
# quantes vegades es passa cada aula dels limits marcats a dalt (comfort i una mica més)
qualitat_aules.head()

Aula  Month  temp_excessiva_alta  temp_excessiva_baixa  temp_excessiva  \
0  A-11      1                 0.00                 91.94           91.94   
1  A-11      2                 0.00                 63.84           63.84   
2  A-11      3                 0.00                 27.05           27.05   
3  A-11      4                 0.28                 35.14           35.42   
4  A-11      5                 8.33                  0.54            8.87   

   co2_excessiu  humitat_excessiva  humitat_alta  humitat_baixa  
0          0.00              31.45          2.15          29.30  
1          3.57              20.98          0.15          20.83  
2          2.02               9.29          0.00           9.29  
3          1.25               3.75          0.83           2.92  
4          0.81               1.21          0.94           0.27

In [610]:
aules = pd.DataFrame(data={'Aula': ['A-11','A-12','A-13','A-14','A-21','A-22','A-23','A-24','A-31','A-32',
'A-33','A-34','A-35','A-36','A-41','A-42','A-43','A-44','A-51','A-52','A-53','A-54','A-55','A-56','A-61','A-62'],
                               'planta': [1,1,1,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,5,5,5,5,5,5,6,6],
                               'posicio': [1,2,3,4,1,2,3,4,1,2,3,4,5,6,1,2,3,4,1,2,3,4,5,6,1,2]
})

In [611]:
source = pd.merge(qualitat_aules, aules, on=['Aula'])
source.to_csv('qualitat_aules.csv')

In [612]:
import altair as alt

# Configure heatmap
heatmap = alt.Chart(source).mark_rect().encode(
    x=alt.X('posicio:O'),
    y=alt.Y('planta:O', scale=alt.Scale(reverse=True)),
    color = 'temp_excessiva:Q'
)

# Configure text
text = alt.Chart(source).mark_text(baseline='middle').encode(
    x=alt.X('posicio:O'),
    y=alt.Y('planta:O', scale=alt.Scale(reverse=True)),
    text = alt.Text('Aula:N'),
)

# Draw the chart
chart = heatmap + text
chart.properties(width = 400, height = 400)

alt.LayerChart(...)

In [613]:
source.head()

Aula  Month  temp_excessiva_alta  temp_excessiva_baixa  temp_excessiva  \
0  A-11      1                 0.00                 91.94           91.94   
1  A-11      2                 0.00                 63.84           63.84   
2  A-11      3                 0.00                 27.05           27.05   
3  A-11      4                 0.28                 35.14           35.42   
4  A-11      5                 8.33                  0.54            8.87   

   co2_excessiu  humitat_excessiva  humitat_alta  humitat_baixa  planta  \
0          0.00              31.45          2.15          29.30       1   
1          3.57              20.98          0.15          20.83       1   
2          2.02               9.29          0.00           9.29       1   
3          1.25               3.75          0.83           2.92       1   
4          0.81               1.21          0.94           0.27       1   

   posicio  
0        1  
1        1  
2        1  
3        1  
4        1

In [614]:
#result['Date'] = pd.to_datetime(result['Date'], format='%Y-%m-%d %H:%M:%S')
#result['epoch'] = result['Date'].dt.month.map(epochs_mapping)

In [615]:
result.head()

Date  Aula  Temperatura  max_comfort  min_comfort    CO2  \
0  2023-01-01 00:00:00  A-11        17.80           25           21  492.5   
1  2023-01-01 01:00:00  A-11        17.70           25           21  511.0   
2  2023-01-01 02:00:00  A-11        17.60           25           21  522.0   
3  2023-01-01 03:00:00  A-11        17.50           25           21  564.0   
4  2023-01-01 04:00:00  A-11        17.45           25           21  521.0   

   outdoors_co2  max_co2  Humidity  min_humidity  max_humidity  \
0           400     1000      55.0            30            60   
1           400     1000      55.0            30            60   
2           400     1000      55.0            30            60   
3           400     1000      54.5            30            60   
4           400     1000      54.0            30            60   

   temp_excessiva_alta  temp_excessiva_baixa  temp_excessiva  co2_excessiu  \
0                False                  True            True         False   
1                False                  True            True         False   
2                False                  True            True         False   
3                False                  True            True         False   
4                False                  True            True         False   

   humitat_excessiva  humitat_alta  humitat_baixa  
0              False         False          False  
1              False         False          False  
2              False         False          False  
3              False         False          False  
4              False         False          False

In [616]:
# GET QUALITY MEASURES
# Remove days vacaion and hours
dies_festius = ['2023-09-25', '2023-10-12', '2023-11-01', '2023-12-06', '2023-12-07', '2023-12-08',
              '2023-12-23','2023-12-24','2023-12-25','2023-12-26','2023-12-27','2023-12-28','2023-12-29',
              '2023-12-30', '2023-12-31','2023-01-01','2023-01-02','2023-01-03','2023-01-04','2023-01-05',
              '2023-01-06','2023-01-07','2023-01-08','2023-01-09','2023-01-10','2023-04-01','2023-04-02',
              '2023-04-03','2023-04-04','2023-04-05','2023-04-06','2023-04-07','2023-04-08','2023-04-09',
              '2023-04-10','2023-05-01','2023-09-01','2023-09-02','2023-09-03','2023-09-04','2023-09-05',
               '2023-09-06','2023-09-07','2023-09-08','2023-09-09','2023-09-10','2023-09-11','2023-09-12']

def marcar_festius_seasonalplot(date):
    return (str(date).split(' ')[0] in dies_festius) or (str(date).split('-')[1] == '08') or (str(date).split('-')[1] == '07')
result['Date'] = pd.to_datetime(result['Date'], format='%Y-%m-%d %H:%M:%S')
result['Month'] = result['Date'].dt.month
result['Hour'] = result['Date'].dt.hour
result = result[(result['Hour']<=21) & (result['Hour']>=8)]
result['vacation'] = result['Date'].map(marcar_festius_seasonalplot)
result = result[result['vacation']==False]
result['day_of_week'] = result['Date'].dt.weekday
# Create a new column to indicate if the date falls on a weekend (Saturday or Sunday)
result['is_weekend'] = result['day_of_week'].isin([5, 6])
result = result[result['is_weekend']==False]
sourceQ1 = result[result['Month'].isin([9,10,11,12,1])]
sourceQ2 = result[result['Month'].isin([2,3,4,5,6])]


# Define comfort zones
comfort_zones = {
    'Temperature': (20, 25),
    'Humidity': (30, 60),
    'CO2': (0, 1000)
}

# Function to calculate deviation from comfort zone
def calculate_deviation(value, lower_bound, upper_bound):
    if value < lower_bound:
        return lower_bound - value
    elif value > upper_bound:
        return value - upper_bound
    else:
        return 0

# Calculate deviations for each measurement
result['Temperature_Deviation'] = result['Temperatura'].apply(calculate_deviation, args=comfort_zones['Temperature'])
result['Humidity_Deviation'] = result['Humidity'].apply(calculate_deviation, args=comfort_zones['Humidity'])
result['CO2_Deviation'] = result['CO2'].apply(calculate_deviation, args=comfort_zones['CO2'])

sourceQ1['Temperature_Deviation'] = sourceQ1['Temperatura'].apply(calculate_deviation, args=comfort_zones['Temperature'])
sourceQ1['Humidity_Deviation'] = sourceQ1['Humidity'].apply(calculate_deviation, args=comfort_zones['Humidity'])
sourceQ1['CO2_Deviation'] = sourceQ1['CO2'].apply(calculate_deviation, args=comfort_zones['CO2'])

sourceQ2['Temperature_Deviation'] = sourceQ2['Temperatura'].apply(calculate_deviation, args=comfort_zones['Temperature'])
sourceQ2['Humidity_Deviation'] = sourceQ2['Humidity'].apply(calculate_deviation, args=comfort_zones['Humidity'])
sourceQ2['CO2_Deviation'] = sourceQ2['CO2'].apply(calculate_deviation, args=comfort_zones['CO2'])

result = result.drop(columns=['Temperatura','max_comfort','min_comfort','CO2','outdoors_co2','max_co2','Humidity','min_humidity','max_humidity'])
sourceQ1 = sourceQ1.drop(columns=['Temperatura','max_comfort','min_comfort','CO2','outdoors_co2','max_co2','Humidity','min_humidity','max_humidity'])
sourceQ2 = sourceQ2.drop(columns=['Temperatura','max_comfort','min_comfort','CO2','outdoors_co2','max_co2','Humidity','min_humidity','max_humidity'])
# Group by class and calculate mean deviation for each class
class_quality = result.groupby('Aula').agg({
    'Temperature_Deviation': 'mean',
    'Humidity_Deviation': 'mean',
    'CO2_Deviation': 'mean'
}).reset_index()
class_qualityQ1 = sourceQ1.groupby('Aula').agg({
    'Temperature_Deviation': 'mean',
    'Humidity_Deviation': 'mean',
    'CO2_Deviation': 'mean'
}).reset_index()
class_qualityQ2 = sourceQ2.groupby('Aula').agg({
    'Temperature_Deviation': 'mean',
    'Humidity_Deviation': 'mean',
    'CO2_Deviation': 'mean'
}).reset_index()

# Normalize deviations to get a global quality measure
class_quality['Temperature_Quality'] = class_quality['Temperature_Deviation'] / (comfort_zones['Temperature'][1] - comfort_zones['Temperature'][0])
class_quality['Humidity_Quality'] = class_quality['Humidity_Deviation'] / (comfort_zones['Humidity'][1] - comfort_zones['Humidity'][0])
class_quality['CO2_Quality'] = class_quality['CO2_Deviation'] / (comfort_zones['CO2'][1] - comfort_zones['CO2'][0])

class_qualityQ1['Temperature_Quality'] = class_qualityQ1['Temperature_Deviation'] / (comfort_zones['Temperature'][1] - comfort_zones['Temperature'][0])
class_qualityQ1['Humidity_Quality'] = class_qualityQ1['Humidity_Deviation'] / (comfort_zones['Humidity'][1] - comfort_zones['Humidity'][0])
class_qualityQ1['CO2_Quality'] = class_qualityQ1['CO2_Deviation'] / (comfort_zones['CO2'][1] - comfort_zones['CO2'][0])

class_qualityQ2['Temperature_Quality'] = class_qualityQ2['Temperature_Deviation'] / (comfort_zones['Temperature'][1] - comfort_zones['Temperature'][0])
class_qualityQ2['Humidity_Quality'] = class_qualityQ2['Humidity_Deviation'] / (comfort_zones['Humidity'][1] - comfort_zones['Humidity'][0])
class_qualityQ2['CO2_Quality'] = class_qualityQ2['CO2_Deviation'] / (comfort_zones['CO2'][1] - comfort_zones['CO2'][0])

# Calculate global quality measure (average of normalized deviations)
class_quality['Global_Quality'] = class_quality[['Temperature_Quality', 'Humidity_Quality', 'CO2_Quality']].mean(axis=1)
class_qualityQ1['Global_Quality'] = class_qualityQ1[['Temperature_Quality', 'Humidity_Quality', 'CO2_Quality']].mean(axis=1)
class_qualityQ2['Global_Quality'] = class_qualityQ2[['Temperature_Quality', 'Humidity_Quality', 'CO2_Quality']].mean(axis=1)

<ipython-input-616-d8dec56c2420>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sourceQ1['Temperature_Deviation'] = sourceQ1['Temperatura'].apply(calculate_deviation, args=comfort_zones['Temperature'])
<ipython-input-616-d8dec56c2420>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sourceQ1['Humidity_Deviation'] = sourceQ1['Humidity'].apply(calculate_deviation, args=comfort_zones['Humidity'])
<ipython-input-616-d8dec56c2420>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [617]:
class_quality.head()

Aula  Temperature_Deviation  Humidity_Deviation  CO2_Deviation  \
0  A-11               0.815483            0.533763       7.336651   
1  A-12               0.862542            0.389735       6.141881   
2  A-13               0.864344            0.318306      11.012685   
3  A-14               0.896624            0.280835       6.073575   
4  A-21               0.943332            0.511970       3.023614   

   Temperature_Quality  Humidity_Quality  CO2_Quality  Global_Quality  
0             0.163097          0.017792     0.007337        0.062742  
1             0.172508          0.012991     0.006142        0.063880  
2             0.172869          0.010610     0.011013        0.064831  
3             0.179325          0.009361     0.006074        0.064920  
4             0.188666          0.017066     0.003024        0.069585

In [618]:
aules = pd.DataFrame(data={'Aula': ['A-11','A-12','A-13','A-14','A-21','A-22','A-23','A-24','A-31','A-32',
'A-33','A-34','A-35','A-36','A-41','A-42','A-43','A-44','A-51','A-52','A-53','A-54','A-55','A-56','A-61','A-62'],
                               'planta': [1,1,1,1,2,2,2,2,3,3,3,3,3,3,4,4,4,4,5,5,5,5,5,5,6,6],
                               'posicio': [1,2,3,4,1,2,3,4,1,2,3,4,5,6,1,2,3,4,1,2,3,4,5,6,1,2]
})
class_quality = pd.merge(class_quality, aules, on=['Aula'])
class_qualityQ1 = pd.merge(class_qualityQ1, aules, on=['Aula'])
class_qualityQ2 = pd.merge(class_qualityQ2, aules, on=['Aula'])

class_quality.head()

Aula  Temperature_Deviation  Humidity_Deviation  CO2_Deviation  \
0  A-11               0.815483            0.533763       7.336651   
1  A-12               0.862542            0.389735       6.141881   
2  A-13               0.864344            0.318306      11.012685   
3  A-14               0.896624            0.280835       6.073575   
4  A-21               0.943332            0.511970       3.023614   

   Temperature_Quality  Humidity_Quality  CO2_Quality  Global_Quality  planta  \
0             0.163097          0.017792     0.007337        0.062742       1   
1             0.172508          0.012991     0.006142        0.063880       1   
2             0.172869          0.010610     0.011013        0.064831       1   
3             0.179325          0.009361     0.006074        0.064920       1   
4             0.188666          0.017066     0.003024        0.069585       2   

   posicio  
0        1  
1        2  
2        3  
3        4  
4        1

**METRIQUES QUALITY**

To help you achieve the desired quality metrics without repeating your existing code, let's focus on creating new metrics for:

1. Deviation from Comfort Zone: Calculating the extent to which the temperature, humidity, and CO2 levels deviate from their respective comfort zones.
2. Global Quality Measure: Combining the individual quality metrics for temperature, humidity, and CO2 into a single global quality measure for each class.

In [619]:
class_quality.to_csv('class_quality_measures.csv')
class_qualityQ1.to_csv('class_quality_measuresQ1.csv')
class_qualityQ2.to_csv('class_quality_measuresQ2.csv')

In [416]:
source = pd.read_csv('/home/olga/Desktop/tfg/qualitat_aules.csv')

slider_month = alt.binding_range(name='Month', min=1, max=12, step=1)
selector_month = alt.selection_single(name="Select a month", fields=['Month'], bind=slider_month, init={'Month': 1})

# TEMPERATURA ALTA
# Configure heatmap
heatmap = alt.Chart(source).mark_rect(color = 'red').encode(
    x=alt.X('posicio:O'),
    y=alt.Y('planta:O', scale=alt.Scale(reverse=True)),
    color = alt.Color('temp_excessiva_alta:Q', scale=alt.Scale(scheme='reds'))
)

# Configure text
text = alt.Chart(source).mark_text(baseline='middle').encode(
    x=alt.X('posicio:O'),
    y=alt.Y('planta:O', scale=alt.Scale(reverse=True)),
    text = alt.Text('Aula:N'),
    color = alt.condition(alt.datum.temp_excessiva_alta > 31, alt.value('white'),alt.value('black'))
)

# Draw the chart
chart1 = (heatmap + text)

# TEMPERATURA BAIXA
# Configure heatmap
heatmap2 = alt.Chart(source).mark_rect(color = 'red').encode(
    x=alt.X('posicio:O'),
    y=alt.Y('planta:O', scale=alt.Scale(reverse=True)),
    color = alt.Color('temp_excessiva_baixa:Q')
)

# Configure text
text2 = alt.Chart(source).mark_text(baseline='middle').encode(
    x=alt.X('posicio:O'),
    y=alt.Y('planta:O', scale=alt.Scale(reverse=True)),
    text = alt.Text('Aula:N'),
    color = alt.condition(alt.datum.temp_excessiva_baixa > 8, alt.value('white'),alt.value('black'))
)

# Draw the chart
chart2 = (heatmap2 + text2)

chart = (chart1 | chart2).add_selection(selector_month).transform_filter(selector_month)
chart

alt.HConcatChart(...)

In [470]:
source.head()

Aula  Month  temp_excessiva_alta  temp_excessiva_baixa  temp_excessiva  \
0  A-11      1                 0.00                 91.94           91.94   
1  A-11      2                 0.00                 63.84           63.84   
2  A-11      3                 0.00                 27.05           27.05   
3  A-11      4                 0.28                 35.14           35.42   
4  A-11      5                 8.33                  0.54            8.87   

   co2_excessiu  humitat_excessiva  humitat_alta  humitat_baixa  planta  \
0          0.00              31.45          2.15          29.30       1   
1          3.57              20.98          0.15          20.83       1   
2          2.02               9.29          0.00           9.29       1   
3          1.25               3.75          0.83           2.92       1   
4          0.81               1.21          0.94           0.27       1   

   posicio  
0        1  
1        1  
2        1  
3        1  
4        1

In [442]:
# create pie_data

In [477]:
# Group by 'Aula' and 'Month' and calculate percentages
grouped = source.groupby(['Aula', 'Month']).mean().reset_index()

# Calculate percentages for each zone
groupedt = grouped.copy()
groupedt = groupedt[['Aula','Month','temp_excessiva_baixa','temp_excessiva_alta']]
groupedt['Comfort'] = 100 - (groupedt['temp_excessiva_baixa'] + groupedt['temp_excessiva_alta'])
groupedt = groupedt.rename(columns={'temp_excessiva_baixa':'Lower', 'temp_excessiva_alta':'Upper'})

groupedco = grouped.copy()
groupedco = groupedco[['Aula','Month','co2_excessiu']]
groupedco['Comfort'] = 100 - groupedco['co2_excessiu']
groupedco = groupedco.rename(columns={'co2_excessiu':'Upper'})
groupedco['Lower'] = 0

groupedh = grouped.copy()
groupedh = groupedh[['Aula','Month','humitat_baixa','humitat_alta']]
groupedh['Comfort'] = 100 - (groupedh['humitat_baixa'] + groupedh['humitat_alta'])
groupedh = groupedh.rename(columns={'humitat_baixa':'Lower', 'humitat_alta':'Upper'})


In [472]:
groupedt.head()

Aula  Month  Lower  Upper  Comfort
0  A-11      1  91.94   0.00     8.06
1  A-11      2  63.84   0.00    36.16
2  A-11      3  27.05   0.00    72.95
3  A-11      4  35.14   0.28    64.58
4  A-11      5   0.54   8.33    91.13

In [460]:
groupedt.to_csv('piequalitytemp.csv')
groupedco.to_csv('piequalityco2.csv')
groupedh.to_csv('piequalityhum.csv')


In [478]:
groupedt=groupedt.drop(columns=['Month'])
groupedt = groupedt.groupby('Aula').mean().reset_index()
melted_df = groupedt.melt(id_vars=['Aula'], var_name='Zone', value_name='Percentage')
melted_df.head()

Aula   Zone  Percentage
0  A-11  Lower   24.888333
1  A-12  Lower   26.199167
2  A-13  Lower   25.677500
3  A-14  Lower   30.019167
4  A-21  Lower   20.361667

In [479]:
check100_df = melted_df.groupby(['Aula'])['Percentage'].sum().reset_index()
check100_df[check100_df['Percentage']!=100]

Empty DataFrame
Columns: [Aula, Percentage]
Index: []

In [480]:
# PIE CHARTSSS
#groupedt = groupedt.groupby('Aula').mean().reset_index()
#melted_df = groupedt.melt(id_vars=['Aula'], var_name='Zone', value_name='Percentage')

# Create hierarchical pie chart
pie_chart = alt.Chart(melted_df).mark_arc(outerRadius=100).encode(
    theta=alt.Theta('Percentage:Q'),
    color=alt.Color('Zone:N', scale=alt.Scale(domain=['Lower', 'Upper', 'Comfort'], range=['#FF5733', '#33FF57', '#3399FF'])),
    tooltip=['Percentage:Q', 'Zone:N']
).properties(
    width = 150,
    height = 150
).facet(column = 'Aula:N', spacing = 20).resolve_scale(theta="independent")

In [465]:
pie_chart

alt.FacetChart(...)

In [ ]:
print()